<a href="https://colab.research.google.com/github/salmenhsairi/EndOfStudiesProjectNotebooks/blob/main/TrainingLayoutlmV2_Colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Mount drive fs

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## Installing requirements

In [ ]:
! pip install -q git+https://github.com/huggingface/transformers.git
! pip install - q git+https: // github.com/huggingface/datasets.git "dill<0.3.5" seqeval
! python -m pip install -q 'git+https://github.com/facebookresearch/detectron2.git'

In [ ]:
! rm -r layoutlmv2_fine_tuning
! git clone -b main https://github.com/salmenhsairi/layoutlmv2_fine_tuning.git

rm: cannot remove 'layoutlmv2_fine_tuning': No such file or directory
Cloning into 'layoutlmv2_fine_tuning'...
remote: Enumerating objects: 60, done.
remote: Counting objects: 100% (60/60), done.
remote: Compressing objects: 100% (45/45), done.
remote: Total 60 (delta 17), reused 58 (delta 15), pack-reused 0
Unpacking objects: 100% (60/60), done.


## Loading __UBIAI__ BIOES Data zip from Drive

In [ ]:
#!/bin/bash
IOB_DATA_PATH = "/content/drive/MyDrive/NERDatasets/EPIC_Case_Times_Extraction_ddvqdMl.zip"
! cd /content/
! rm -r data
! mkdir data
! cp "$IOB_DATA_PATH" data/dataset.zip 
! cd data && unzip -q dataset && rm dataset.zip
! cd ..

rm: cannot remove 'data': No such file or directory


## Defining preprocessing params and running the preprocessing script

In [ ]:
#!/bin/bash
#preprocessing args
TEST_SIZE = 0.3333333
DATA_OUTPUT_PATH = "/content/"

In [ ]:
! python3 layoutlmv2_fine_tuning/preprocess.py --valid_size $TEST_SIZE --output_path $DATA_OUTPUT_PATH

100% 1/1 [00:00<00:00,  1.12ba/s]
Downloading: 100% 135/135 [00:00<00:00, 90.2kB/s]
Downloading: 100% 226k/226k [00:00<00:00, 1.85MB/s]
Downloading: 100% 707/707 [00:00<00:00, 580kB/s]
100% 1/1 [00:09<00:00,  9.16s/ba]
100% 1/1 [00:11<00:00, 11.61s/ba]


## Proceed with Training

In [ ]:
from transformers import AutoModelForTokenClassification, TrainingArguments, Trainer,AutoTokenizer
from datasets import load_metric,load_from_disk
import os
import numpy as np
import torch
import warnings

In [ ]:
# load datasets
train_dataset = load_from_disk(f'/content/train_split')
valid_dataset = load_from_disk(f'/content/eval_split')

In [ ]:
labels = train_dataset.features["labels"].feature.names
num_labels = len(labels)
label2id, id2label = dict(), dict()
for i, label in enumerate(labels):
    label2id[label] = i
    id2label[i] = label

In [ ]:
# Metrics
metric = load_metric("seqeval")
return_entity_level_metrics = False

def compute_metrics(p):
    predictions, labels = p
    predictions = np.argmax(predictions, axis=2)

    # Remove ignored index (special tokens)
    true_predictions = [
        [id2label[p] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    true_labels = [
        [id2label[l] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]

    results = metric.compute(predictions=true_predictions, references=true_labels)
    if return_entity_level_metrics:
        # Unpack nested dictionaries
        final_results = {}
        for key, value in results.items():
            if isinstance(value, dict):
                for n, v in value.items():
                    final_results[f"{key}_{n}"] = v
            else:
                final_results[key] = value
        return final_results
    else:
        return {
            "precision": results["overall_precision"],
            "recall": results["overall_recall"],
            "f1": results["overall_f1"],
            "accuracy": results["overall_accuracy"],
        }

### loading model and preprocessor (also required for Hugging face trainer)

In [ ]:
model = AutoModelForTokenClassification.from_pretrained('microsoft/layoutlmv2-base-uncased',
                                                                    num_labels=len(label2id))

tokenizer = AutoTokenizer.from_pretrained('microsoft/layoutlmv2-base-uncased')

# Set id2label and label2id 
model.config.id2label = id2label
model.config.label2id = label2id

Downloading:   0%|          | 0.00/765M [00:00<?, ?B/s]

Some weights of the model checkpoint at microsoft/layoutlmv2-base-uncased were not used when initializing LayoutLMv2ForTokenClassification: ['layoutlmv2.visual.backbone.bottom_up.res4.19.conv3.norm.num_batches_tracked', 'layoutlmv2.visual.backbone.bottom_up.res4.15.conv3.norm.num_batches_tracked', 'layoutlmv2.visual.backbone.bottom_up.res4.15.conv1.norm.num_batches_tracked', 'layoutlmv2.visual.backbone.bottom_up.res4.6.conv2.norm.num_batches_tracked', 'layoutlmv2.visual.backbone.bottom_up.res5.1.conv1.norm.num_batches_tracked', 'layoutlmv2.visual.backbone.bottom_up.res4.20.conv3.norm.num_batches_tracked', 'layoutlmv2.visual.backbone.bottom_up.res3.3.conv3.norm.num_batches_tracked', 'layoutlmv2.visual.backbone.bottom_up.res4.10.conv2.norm.num_batches_tracked', 'layoutlmv2.visual.backbone.bottom_up.res5.2.conv1.norm.num_batches_tracked', 'layoutlmv2.visual.backbone.bottom_up.res4.11.conv2.norm.num_batches_tracked', 'layoutlmv2.visual.backbone.bottom_up.res2.2.conv1.norm.num_batches_track

Downloading:   0%|          | 0.00/226k [00:00<?, ?B/s]

### train the model

In [ ]:
args = TrainingArguments(
    output_dir='test', # name of directory to store the checkpoints
    # max_steps=2500
    evaluation_strategy = "epoch",
    logging_strategy = "epoch",
    num_train_epochs=10,
    learning_rate = 5e-5,
    per_device_train_batch_size=1,
    per_device_eval_batch_size=1,
    metric_for_best_model = "f1",
    # lr_scheduler_type = LR_SCHEDULER_TYPE,  #constant/linear/cosine/cosine_with_restarts/polynomial/constant_with_warmup
    # warmup_ratio = WARMUP_RATIO, # optional, defaults to 0.0
    # max_steps=EPOCHS*len(train_dataloader),
    # fp16=True, # we use mixed precision (less memory consumption)
    save_total_limit = 1,
    load_best_model_at_end=True,
    save_strategy = "epoch")

In [ ]:
trainer = Trainer(
    model=model,
    args=args,
    train_dataset=train_dataset,
    eval_dataset=valid_dataset,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

In [ ]:
# trainer.train()

evaluate the model

In [ ]:
trainer.evaluate()

### save the model for upcoming fine-tuning/infrence

In [ ]:
trainer.save_model('/content/drive/MyDrive/Colab Notebooks/Work/LayoutLMModel/TestLayoutLMV2')

In [ ]:
import gc
gc.collect()
# ! rm -r layoutlmv2-finetuned

88